In [1]:
%%writefile requirements.txt
langchain_community
langchain
langchain_google_genai
loguru
sentence-transformers==2.3.0
chromadb
matplotlib
jsonargparse
langchain-openai
pymupdf
langchain-core
langchain-community
chromadb
beautifulsoup4
requests
python-telegram-bot
tqdm
pymupdf

Overwriting requirements.txt


In [2]:
!pip install -r requirements.txt

In [4]:
import os

In [5]:
if not os.path.exists("./data"):
  os.mkdir("./data")

PDF-files preprocessing and chunking

In [16]:
import fitz
from loguru import logger
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter


def clean_extra_whitespace(text):
    return ' '.join(text.split())


def group_paragraphs(text):
    return text.replace("\n", " ").replace("\r", " ")


def load_pdf(files):
    if not isinstance(files, list):
        files = [files]

    documents = []
    for file_path in files:
        try:
            logger.info(f"Загрузка PDF: {file_path}")
            doc = fitz.open(file_path)
            text = ""

            for page_num in range(len(doc)):
                page = doc.load_page(page_num)
                text += page.get_text("text")

            text = clean_extra_whitespace(text)
            text = group_paragraphs(text)

            document = Document(
                page_content=text,
                metadata={"source": file_path}
            )
            documents.append(document)
            logger.success(f"Успешно загружен: {file_path}")

        except Exception as e:
            logger.error(f"Ошибка загрузки {file_path}: {e}")
            raise

    return documents


def chunk_documents(documents):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1200,
        chunk_overlap=150,
        separators=["\n\n## ", "\n\n# ", "\n\n", "\n", ". "]
    )

    chunks = splitter.split_documents(documents)
    analyze_chunk_quality(chunks)
    return chunks


def analyze_chunk_quality(chunks):
    if not chunks:
        return

    chunk_sizes = [len(chunk.page_content) for chunk in chunks]
    avg_size = sum(chunk_sizes) / len(chunk_sizes)

    logger.info("Статистика чанков:")
    logger.info(f"   • Общее количество: {len(chunks)}")
    logger.info(f"   • Средний размер: {avg_size:.0f} символов")
    logger.info(f"   • Минимальный: {min(chunk_sizes)}")
    logger.info(f"   • Максимальный: {max(chunk_sizes)}")

    good_chunks = [c for c in chunks if 200 < len(c.page_content) < 1500]
    quality_ratio = len(good_chunks) / len(chunks)

    logger.info(f"   • Качество: {quality_ratio:.1%} хороших чанков")


In [17]:
directory = "./data"
files = os.listdir(directory)
pdf_files = [f"{directory}/{file}" for file in files if file.endswith('.pdf')]
print(pdf_files)
documents = load_pdf(files=pdf_files)
chunks = chunk_documents(documents)
analyze_chunk_quality(chunks)

2025-10-22 18:22:43.748 | INFO     | __main__:load_pdf:22 - Загрузка PDF: ./data/Sinners_Kaenriah.pdf
2025-10-22 18:22:43.817 | SUCCESS  | __main__:load_pdf:38 - Успешно загружен: ./data/Sinners_Kaenriah.pdf
2025-10-22 18:22:43.819 | INFO     | __main__:load_pdf:22 - Загрузка PDF: ./data/Natlan.pdf


['./data/Sinners_Kaenriah.pdf', './data/Natlan.pdf']


2025-10-22 18:22:44.173 | SUCCESS  | __main__:load_pdf:38 - Успешно загружен: ./data/Natlan.pdf
2025-10-22 18:22:44.191 | INFO     | __main__:analyze_chunk_quality:66 - Статистика чанков:
2025-10-22 18:22:44.191 | INFO     | __main__:analyze_chunk_quality:67 -    • Общее количество: 151
2025-10-22 18:22:44.193 | INFO     | __main__:analyze_chunk_quality:68 -    • Средний размер: 1122 символов
2025-10-22 18:22:44.195 | INFO     | __main__:analyze_chunk_quality:69 -    • Минимальный: 610
2025-10-22 18:22:44.196 | INFO     | __main__:analyze_chunk_quality:70 -    • Максимальный: 1199
2025-10-22 18:22:44.198 | INFO     | __main__:analyze_chunk_quality:75 -    • Качество: 100.0% хороших чанков
2025-10-22 18:22:44.200 | INFO     | __main__:analyze_chunk_quality:66 - Статистика чанков:
2025-10-22 18:22:44.204 | INFO     | __main__:analyze_chunk_quality:67 -    • Общее количество: 151
2025-10-22 18:22:44.206 | INFO     | __main__:analyze_chunk_quality:68 -    • Средний размер: 1122 символов
20

Load embedding model and create vectorstore

In [25]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

def load_embedding_model():

    try:
        model_kwargs = {"device": "cpu"}
        encode_kwargs = {"normalize_embeddings": True}

        embedding_model = HuggingFaceEmbeddings(
            model_name='cointegrated/rubert-tiny2',
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs,
        )

        logger.success(f"✅ Модель эмбеддингов загружена: {'cointegrated/rubert-tiny2'}")
        return embedding_model

    except Exception as e:
        logger.error(f"❌ Ошибка загрузки модели: {e}")
        raise


def create_vectorstore(chunks, embedding_model):
        vectorstore = Chroma.from_documents(
            documents=chunks,
            embedding=embedding_model
        )

        return vectorstore

def search_similar(vectorstore, query, k, filter_dict=None):

    results = vectorstore.similarity_search(
        query=query,
        k=k,
        filter=filter_dict
    )

    logger.info(f"Найдено {len(results)} результатов для: '{query}'")
    return results


def retrieve_context(vectorstore, query, k):
    retrieved_docs = search_similar(vectorstore, query, k)
    return retrieved_docs

In [19]:
embedding_model = load_embedding_model()
vectorstore = create_vectorstore(chunks, embedding_model)

/tmp/ipython-input-1335281124.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2025-10-22 18:23:40.835 | SUCCESS  | __main__:load_embedding_model:17 - ✅ Модель эмбеддингов загружена: cointegrated/rubert-tiny2


Load LLM Gemini

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from google.colab import userdata

GEMINI_API_KEY = userdata.get('gemini')

def create_gemini_llm():

    try:
        llm = ChatGoogleGenerativeAI(
            model='gemini-flash-latest',
            temperature=0.3,
            max_tokens=3000,
            google_api_key=GEMINI_API_KEY
        )
        logger.success(f"✅ Gemini модель загружена: {'gemini-flash-latest'}")
        return llm
    except Exception as e:
        logger.error(f"❌ Ошибка загрузки Gemini: {e}")
        return None


def create_prompt_template():
    template = """
Ты - эксперт по лору игры Genshin Impact. Ответь на вопрос пользователя на основе предоставленного контекста.

КОНТЕКСТ:
{context}

ВОПРОС:
{question}

ИНСТРУКЦИИ:
- Отвечай ТОЛЬКО на русском языке
- Используй информацию из предоставленного контекста
- Давай полный развернутый ответ
- Если в контексте нет информации, сообщи об этом

ОТВЕТ:
"""
    return ChatPromptTemplate.from_template(template)


def create_chain():
    gemini_llm = create_gemini_llm()

    prompt = create_prompt_template()
    chain = prompt | gemini_llm | StrOutputParser()
    return chain


def generate_response(question, context):

    try:
        chain = create_chain()

        context_text = "\n\n".join([doc.page_content for doc in context])

        response = chain.invoke({
            "context": context_text,
            "question": question
        })

        return response

    except Exception as e:
        logger.error(f"❌ Ошибка генерации ответа: {e}")
        return "Извините, произошла ошибка при генерации ответа."

In [26]:
question = "Расскажи о героях Каэнри ах"
context = retrieve_context(vectorstore, question, 15)
response = generate_response(question, context)

2025-10-22 18:26:20.213 | INFO     | __main__:search_similar:41 - Найдено 15 результатов для: 'Расскажи о героях Каэнри ах'
2025-10-22 18:26:20.286 | SUCCESS  | __main__:create_gemini_llm:17 - ✅ Gemini модель загружена: gemini-flash-latest


In [27]:
print(response)

В сюжетном квесте 4.7 «Сказка на ночь» Дайнслейф рассказал о шести «героях» Каэнри'ах, которых в настоящее время прозвали «грешниками».

### Общая информация о героях

1.  **Статус и роль:** Все они были великими людьми, лидерами среди своих коллег в различных областях (например, Дайнслейф был лучшим среди мечников, а Рэйндоттир — лучшей среди алхимиков). Королевство Каэнри'ах возлагало на них большие надежды.
2.  **Миссия:** Их конкретная миссия состояла в том, чтобы объединиться и предотвратить грядущую катастрофу — помешать некоему «Чёрному королю» продолжать сотрясать основы мира. Предполагается, что они противостояли существу, связанному со скверной.
3.  **Падение:** В процессе миссии что-то пошло не так. Пять человек (не считая Дайнслейфа), затаивших в своих сердцах собственные желания, не смогли устоять перед искушением «бездны» и разделили силу, которой было достаточно, чтобы уничтожить мир.

### Известные «Герои»/«Грешники»

В контексте упоминаются четыре грешника, а также Дай